In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

In [2]:
col =  pd.read_csv("../collisions2018.csv",dtype={'case_id': str})

C:\Users\Harold\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Utility functions

In [3]:
def extract_set(description):
    lines = description.split("\n")
    trad = {}
    for line in lines:
        l,r = line.split("-",1)
        l= l.strip()
        r= r.strip().lower()
        trad[r]= l
        
    return trad

In [4]:
extract_set("A - Had Not Been Drinking\nB - Had Been Drinking, Under Influence\nC - Had Been Drinking, Not Under Influence\nD - Had Been Drinking, Impairment Unknown\nG - Impairment Unknown\nH - Not Applicable")

{'had not been drinking': 'A',
 'had been drinking, under influence': 'B',
 'had been drinking, not under influence': 'C',
 'had been drinking, impairment unknown': 'D',
 'impairment unknown': 'G',
 'not applicable': 'H'}

In [5]:
def set_translate(trad,s):
    if s in trad:
        return trad[s]
    else:
        return np.nan

## Collision_severity

In [6]:
df = col.copy()

In [7]:
collision_severity_trad = {"fatal":"1","severe injury":"2","other injury":"3","pain":"4","property damage only":"5"}

In [8]:
df['collision_severity'] = df['collision_severity'].apply(lambda s : set_translate(collision_severity_trad,s))

## hit_and_run

In [9]:
hit_and_run_trad = extract_set("F - Felony\nM - Misdemeanor\nN - Not Hit and Run")
df['hit_and_run'] = df['hit_and_run'].apply(lambda s : set_translate(hit_and_run_trad,s))
df.dropna(subset=['hit_and_run'],inplace=True)

Many collision_time are NaN, do we replace by some mean or keep it null or remove the rows ?

## lighting

In [10]:
lighting_trad = {"daylight":"A","dusk or dawn":"B","dark with street lights":"C",
                 "dark with no street lights":"D","dark with street lights not functioning":"E"}

In [11]:
df['lighting'] = df['lighting'].apply(lambda s : set_translate(lighting_trad,s))

## location_type

In [12]:
location_type_trad = extract_set("H - Highway\nI - Intersection\nR - Ramp")


In [13]:
df['location_type'] = df['location_type'].apply(lambda s : set_translate(location_type_trad,s))

## pcf_violation_category

In [14]:
trad="01 - Driving or Bicycling Under the Influence of Alcohol or Drug\n02 - Impeding Traffic\n03 - Unsafe Speed\n04 - Following Too Closely\n05 - Wrong Side of Road\n06 - Improper Passing"
trad2="\n07 - Unsafe Lane Change\n08 - Improper Turning\n09 - Automobile Right of Way\n10 - Pedestrian Right of Way\n11 - Pedestrian Violation\n12 - Traffic Signals and Signs\n13 - Hazardous Parking\n14 - Lights\n15 - Brakes\n16 - Other Equipment\n17 - Other Hazardous Violation\n18 - Other Than Driver (or Pedestrian)\n19 - Unsafe Starting or Backing \n22 - Other Improper Driving\n23 - Pedestrian or \"Other\" Under the Influence of Alcohol or Drug\n24 - Fell Asleep\n00 - Unknown"

trad =trad+trad2
pcf_violation_category_trad = extract_set(trad)
pcf_violation_category_trad['dui']='01'
pcf_violation_category_trad['speeding']='03'
print(pcf_violation_category_trad)

{'driving or bicycling under the influence of alcohol or drug': '01', 'impeding traffic': '02', 'unsafe speed': '03', 'following too closely': '04', 'wrong side of road': '05', 'improper passing': '06', 'unsafe lane change': '07', 'improper turning': '08', 'automobile right of way': '09', 'pedestrian right of way': '10', 'pedestrian violation': '11', 'traffic signals and signs': '12', 'hazardous parking': '13', 'lights': '14', 'brakes': '15', 'other equipment': '16', 'other hazardous violation': '17', 'other than driver (or pedestrian)': '18', 'unsafe starting or backing': '19', 'other improper driving': '22', 'pedestrian or "other" under the influence of alcohol or drug': '23', 'fell asleep': '24', 'unknown': '00', 'dui': '01', 'speeding': '03'}


In [15]:
before = df['pcf_violation_category'].isnull().sum()

In [16]:
df['pcf_violation_category'] = df['pcf_violation_category'].apply(lambda s : set_translate(pcf_violation_category_trad,s))

In [17]:
display(df['pcf_violation_category'].head(50))

0     08
1     13
2     17
3     12
4     03
5     19
6     07
7     03
8     09
9     19
10    17
11    05
12    12
13    03
14    08
15    12
16    01
17    09
18    08
19    08
20    18
21    03
22    08
23    24
24    19
25    03
26    03
27    12
28    06
29    03
30    03
31    03
32    08
33    07
34    08
35    03
36    10
37    03
38    03
39    03
40    03
41    03
42    07
43    07
44    03
45    08
46    03
47    03
48    18
49    03
Name: pcf_violation_category, dtype: object

In [18]:
display(df['pcf_violation_subsection'].head(50))

0     NaN
1       A
2       A
3       A
4     NaN
5     NaN
6       A
7     NaN
8       A
9     NaN
10      A
11      A
12      A
13    NaN
14    NaN
15      A
16      A
17      C
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
26    NaN
27      A
28    NaN
29    NaN
30    NaN
31    NaN
32    NaN
33      A
34    NaN
35    NaN
36      A
37    NaN
38    NaN
39    NaN
40    NaN
41    NaN
42      A
43      A
44    NaN
45    NaN
46    NaN
47    NaN
48    NaN
49    NaN
Name: pcf_violation_subsection, dtype: object

## primary_collision_factor

In [19]:
primary_collision_factor_trad = extract_set("A - Vehicle Code Violation\nB - Other Improper Driving\nC - Other Than Driver\nD - Unknown\nE - Fell Asleep")

In [20]:
df['primary_collision_factor'] = df['primary_collision_factor'].apply(lambda s : set_translate(primary_collision_factor_trad,s))

## road_condition_1 and 2

In [21]:
road_condition_1_trad ={"holes":"A","loose material":"B","obstruction":"C",
                 "construction":"D","reduced width":"E","flooded":"F","other":"G","normal":"H"}



In [22]:
df['road_condition_1'] = df['road_condition_1'].apply(lambda s : set_translate(road_condition_1_trad,s))
df['road_condition_2'] = df['road_condition_2'].apply(lambda s : set_translate(road_condition_1_trad,s))


## road_surface

In [23]:
road_surface_trad = extract_set("A - Dry\nB - Wet\nC - Snowy\nD - Slippery")


In [24]:
df['road_surface'] = df['road_surface'].apply(lambda s : set_translate(road_surface_trad,s))

## type_of_collision

In [25]:
type_of_collision_trad = extract_set("A - Head-On\nB - Sideswipe\nC - Rear End\nD - Broadside\nE - Hit Object\nF - Overturned\nG - Pedestrian\nH - Other")

In [26]:
df['type_of_collision'] = df['type_of_collision'].apply(lambda s : set_translate(type_of_collision_trad,s))

## weather_1

In [27]:
weather_1_trad = extract_set("A - Clear\nB - Cloudy\nC - Raining\nD - Snowing\nE - Fog\nF - Other\nG - Wind")

In [28]:
df['weather_1'] = df['weather_1'].apply(lambda s : set_translate(weather_1_trad,s))
df['weather_2'] = df['weather_2'].apply(lambda s : set_translate(weather_1_trad,s))

##  Changing types

In [29]:
print(df.dtypes)

case_id                      object
collision_date               object
collision_severity           object
collision_time               object
county_city_location          int64
hit_and_run                  object
jurisdiction                float64
lighting                     object
location_type                object
officer_id                   object
pcf_violation               float64
pcf_violation_category       object
pcf_violation_subsection     object
population                  float64
primary_collision_factor     object
process_date                 object
ramp_intersection           float64
road_condition_1             object
road_condition_2             object
road_surface                 object
tow_away                    float64
type_of_collision            object
weather_1                    object
weather_2                    object
dtype: object


In [30]:
df['county_city_location'] = df['county_city_location'].astype(int)

In [31]:
df['jurisdiction'] = df['jurisdiction'].astype('Int64')

In [32]:
df['pcf_violation'] = df['pcf_violation'].astype('Int64')

In [33]:
def float2int2str(x):
    if x != x:
        return x
    else:
        return str(int(x))

In [34]:
df['population'] = df['population'].apply(float2int2str)

In [35]:
df['ramp_intersection'] = df['ramp_intersection'].apply(float2int2str)

In [36]:
df['tow_away'] = df['tow_away'].astype('Int64')

In [37]:
print(df['collision_severity'].isnull().sum())

0


## Export

In [38]:
df.to_csv(r'../collisions_clean.csv',index=False)